In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

import os

from PIL import Image

Using TensorFlow backend.


In [2]:
def load_data():
        x_train = np.load(r'C:\Users\Vee\Desktop\python\answers.npy',allow_pickle=True)
        x_train = x_train.reshape(721,4,4,1)
        return x_train

In [3]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 4
        self.img_cols = 4
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 16

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()
        generator = self.generator

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()
        model.add(Dense(16 * 4 * 4, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((4, 4, 16)))
        model.add(Conv2D(16, kernel_size=(2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(16, kernel_size=(2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(16, kernel_size=(2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(16, kernel_size=(2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(16, kernel_size=(2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(1, kernel_size=(2,2), padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)
    

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        X_train = load_data()

        # Rescale -1 to 1
        X_train = X_train / 256

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                #self.save_imgs(epoch)
                self.generator.save("generator.h5")

    def save_imgs(self, epoch):
        r, c = 3, 3
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        gen_imgs = np.array(gen_imgs) * 256
        gen_imgs = gen_imgs.astype(int)

        fig=plt.figure(figsize=(20, 20))
        for i in range(1, c*r+1):
            img = gen_imgs[i-1]
            fig.add_subplot(r, c, i)
            plt.imshow(img)
        fig.savefig(r"C:\Users\Vee\Desktop\python\GAN\epoch_%d.el
Interrupt
Restart
Restart & Clear Output
Restart & Run All
Reconnect
Shutdown
Change kernel
Widgets
Help
png" % epoch)
        plt.close()
        self.generator.save("generator.h5")

In [4]:
dcgan = DCGAN()
dcgan.train(epochs=1001, batch_size=100, save_interval=100)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 2, 2, 32)          320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 2, 2, 32)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2, 2, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 32)          9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 1, 32)          128       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 1, 1, 32)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 1, 32)         

C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.799277, acc.: 46.00%] [G loss: 0.785889]
1 [D loss: 0.773898, acc.: 51.00%] [G loss: 0.794451]
2 [D loss: 0.782533, acc.: 51.00%] [G loss: 0.834400]
3 [D loss: 0.793022, acc.: 53.00%] [G loss: 0.769421]
4 [D loss: 0.812747, acc.: 47.50%] [G loss: 0.647463]
5 [D loss: 0.778280, acc.: 48.00%] [G loss: 0.821103]


C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


6 [D loss: 0.773078, acc.: 50.50%] [G loss: 0.788113]
7 [D loss: 0.780922, acc.: 49.00%] [G loss: 0.760756]
8 [D loss: 0.718276, acc.: 58.00%] [G loss: 0.691405]
9 [D loss: 0.771391, acc.: 53.00%] [G loss: 0.698243]
10 [D loss: 0.769654, acc.: 50.00%] [G loss: 0.779906]
11 [D loss: 0.746527, acc.: 59.50%] [G loss: 0.729495]
12 [D loss: 0.788640, acc.: 51.00%] [G loss: 0.756235]
13 [D loss: 0.764637, acc.: 54.50%] [G loss: 0.800550]
14 [D loss: 0.754639, acc.: 52.00%] [G loss: 0.754415]
15 [D loss: 0.795344, acc.: 46.00%] [G loss: 0.800321]
16 [D loss: 0.772800, acc.: 52.00%] [G loss: 0.781049]
17 [D loss: 0.746596, acc.: 55.00%] [G loss: 0.683899]
18 [D loss: 0.708969, acc.: 53.50%] [G loss: 0.716281]
19 [D loss: 0.763812, acc.: 52.50%] [G loss: 0.791426]
20 [D loss: 0.746365, acc.: 52.50%] [G loss: 0.719689]
21 [D loss: 0.791690, acc.: 48.50%] [G loss: 0.751164]
22 [D loss: 0.728790, acc.: 56.50%] [G loss: 0.702564]
23 [D loss: 0.735795, acc.: 53.00%] [G loss: 0.788743]
24 [D loss: 0.

...

316 [D loss: 0.460730, acc.: 87.00%] [G loss: 1.066247]
317 [D loss: 0.458567, acc.: 87.50%] [G loss: 1.130818]
318 [D loss: 0.446724, acc.: 86.00%] [G loss: 1.102477]
319 [D loss: 0.430714, acc.: 87.50%] [G loss: 1.118303]
320 [D loss: 0.440261, acc.: 88.00%] [G loss: 1.138975]
321 [D loss: 0.455680, acc.: 87.50%] [G loss: 1.114068]
322 [D loss: 0.452599, acc.: 87.50%] [G loss: 1.104244]
323 [D loss: 0.447211, acc.: 88.50%] [G loss: 1.118085]
324 [D loss: 0.450036, acc.: 88.00%] [G loss: 1.084921]
325 [D loss: 0.426285, acc.: 92.00%] [G loss: 1.205724]
326 [D loss: 0.433250, acc.: 89.00%] [G loss: 1.205963]
327 [D loss: 0.460213, acc.: 87.50%] [G loss: 1.123146]
328 [D loss: 0.466546, acc.: 86.00%] [G loss: 1.156289]
329 [D loss: 0.451644, acc.: 88.00%] [G loss: 1.137854]
330 [D loss: 0.467229, acc.: 85.00%] [G loss: 1.160600]
331 [D loss: 0.424506, acc.: 92.50%] [G loss: 1.188892]
332 [D loss: 0.440074, acc.: 86.00%] [G loss: 1.144649]
333 [D loss: 0.427669, acc.: 88.50%] [G loss: 1.

471 [D loss: 0.391028, acc.: 94.00%] [G loss: 1.399555]
472 [D loss: 0.386853, acc.: 92.50%] [G loss: 1.426423]
473 [D loss: 0.378753, acc.: 94.50%] [G loss: 1.378647]
474 [D loss: 0.373134, acc.: 94.00%] [G loss: 1.490147]
475 [D loss: 0.406595, acc.: 92.00%] [G loss: 1.363448]
476 [D loss: 0.361595, acc.: 95.50%] [G loss: 1.315188]
477 [D loss: 0.376493, acc.: 91.50%] [G loss: 1.404727]
478 [D loss: 0.382540, acc.: 93.00%] [G loss: 1.422559]
479 [D loss: 0.364750, acc.: 95.00%] [G loss: 1.272986]
480 [D loss: 0.382828, acc.: 93.00%] [G loss: 1.420775]
481 [D loss: 0.391095, acc.: 91.00%] [G loss: 1.293686]
482 [D loss: 0.391579, acc.: 92.50%] [G loss: 1.412315]
483 [D loss: 0.390518, acc.: 92.00%] [G loss: 1.451744]
484 [D loss: 0.388096, acc.: 95.00%] [G loss: 1.409824]
485 [D loss: 0.359868, acc.: 93.50%] [G loss: 1.442867]
486 [D loss: 0.395302, acc.: 92.00%] [G loss: 1.404374]
487 [D loss: 0.423665, acc.: 90.00%] [G loss: 1.366713]
488 [D loss: 0.369571, acc.: 95.00%] [G loss: 1.

626 [D loss: 0.371527, acc.: 92.50%] [G loss: 1.520867]
627 [D loss: 0.354194, acc.: 92.00%] [G loss: 1.559188]
628 [D loss: 0.345354, acc.: 94.50%] [G loss: 1.616777]
629 [D loss: 0.335955, acc.: 95.50%] [G loss: 1.491235]
630 [D loss: 0.345091, acc.: 94.00%] [G loss: 1.541338]
631 [D loss: 0.346676, acc.: 94.50%] [G loss: 1.624848]
632 [D loss: 0.341213, acc.: 97.50%] [G loss: 1.542271]
633 [D loss: 0.308920, acc.: 97.00%] [G loss: 1.648833]
634 [D loss: 0.331995, acc.: 95.00%] [G loss: 1.552272]
635 [D loss: 0.330055, acc.: 97.00%] [G loss: 1.568515]
636 [D loss: 0.331194, acc.: 96.00%] [G loss: 1.517568]
637 [D loss: 0.328959, acc.: 95.00%] [G loss: 1.453885]
638 [D loss: 0.332639, acc.: 95.00%] [G loss: 1.642295]
639 [D loss: 0.351562, acc.: 94.00%] [G loss: 1.732192]
640 [D loss: 0.318280, acc.: 96.50%] [G loss: 1.454768]
641 [D loss: 0.337274, acc.: 96.00%] [G loss: 1.634834]
642 [D loss: 0.340509, acc.: 95.50%] [G loss: 1.659031]
643 [D loss: 0.346308, acc.: 94.50%] [G loss: 1.

776 [D loss: 0.313524, acc.: 94.00%] [G loss: 1.678859]
777 [D loss: 0.316440, acc.: 96.00%] [G loss: 1.819476]
778 [D loss: 0.299952, acc.: 95.50%] [G loss: 1.669501]
779 [D loss: 0.296311, acc.: 95.50%] [G loss: 1.848910]
780 [D loss: 0.298529, acc.: 97.00%] [G loss: 1.818185]
781 [D loss: 0.294680, acc.: 95.50%] [G loss: 1.922544]
782 [D loss: 0.316153, acc.: 94.50%] [G loss: 1.687421]
783 [D loss: 0.301421, acc.: 96.00%] [G loss: 1.767333]
784 [D loss: 0.301846, acc.: 95.50%] [G loss: 1.675862]
785 [D loss: 0.304968, acc.: 95.50%] [G loss: 1.664461]
786 [D loss: 0.315857, acc.: 96.50%] [G loss: 1.774170]
787 [D loss: 0.285105, acc.: 98.50%] [G loss: 1.812345]
788 [D loss: 0.300172, acc.: 97.50%] [G loss: 1.736761]
789 [D loss: 0.300893, acc.: 96.00%] [G loss: 1.670276]
790 [D loss: 0.284032, acc.: 96.50%] [G loss: 1.709967]
791 [D loss: 0.309904, acc.: 98.00%] [G loss: 1.801261]
792 [D loss: 0.279286, acc.: 100.00%] [G loss: 1.778568]
793 [D loss: 0.308928, acc.: 95.00%] [G loss: 1

926 [D loss: 0.278107, acc.: 98.00%] [G loss: 1.815347]
927 [D loss: 0.288669, acc.: 97.50%] [G loss: 1.978900]
928 [D loss: 0.291803, acc.: 97.00%] [G loss: 1.884210]
929 [D loss: 0.274264, acc.: 98.00%] [G loss: 1.900158]
930 [D loss: 0.274342, acc.: 98.50%] [G loss: 2.009720]
931 [D loss: 0.274824, acc.: 98.00%] [G loss: 1.908729]
932 [D loss: 0.285546, acc.: 97.50%] [G loss: 2.006296]
933 [D loss: 0.263477, acc.: 99.00%] [G loss: 2.008579]
934 [D loss: 0.286687, acc.: 95.50%] [G loss: 2.002862]
935 [D loss: 0.286750, acc.: 98.00%] [G loss: 1.915098]
936 [D loss: 0.293218, acc.: 96.50%] [G loss: 1.935696]
937 [D loss: 0.262511, acc.: 98.50%] [G loss: 1.959272]
938 [D loss: 0.301820, acc.: 98.00%] [G loss: 1.940667]
939 [D loss: 0.272114, acc.: 98.50%] [G loss: 1.999994]
940 [D loss: 0.262743, acc.: 98.50%] [G loss: 1.914833]
941 [D loss: 0.274169, acc.: 99.00%] [G loss: 1.781168]
942 [D loss: 0.258038, acc.: 98.00%] [G loss: 2.017285]
943 [D loss: 0.267941, acc.: 98.50%] [G loss: 2.